<a href="https://colab.research.google.com/github/BaudraccoTomas/ChatBootInteligente_Alura/blob/main/Chatbot_Inteligente_Alura_Sprint1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Configurar ambiente

In [2]:
!python -m spacy download es_core_news_md
!pip install unidecode
!pip install jellyfish
!pip install transformers

2023-10-09 11:16:45.157296: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 21.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.6 MB/s eta 0:00:00


In [3]:
#Instalando bibliotecas
import pandas as pd
import re, os, random, pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import jellyfish
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch
from unidecode import unidecode
from nltk.tokenize import WhitespaceTokenizer

#Definiendo variables del proyecto:
nlp = spacy.load('es_core_news_md')

#Conectando al Google Drive
from google.colab import drive
drive.mount('/content/drive')
folder = '/content/drive/MyDrive/Chatbot'

Mounted at /content/drive


# 2. Importar verbos

In [4]:
# Importar la lista_verbos:
pickle_file = open("/content/drive/MyDrive/Chatbot/verbos/lista_verbos.pickle", "rb")
lista_verbos = pickle.load(pickle_file)

# Importar el diccionario:
pickle_file = open("/content/drive/MyDrive/Chatbot/verbos/verbos_irregulares.pickle", "rb")
verbos_irregulares = pickle.load(pickle_file)

#3. Tratamiento de datos

In [24]:
#Función para encontrar la raiz de las palabras
def raiz(palabra):
  """
  Funcion para comparar similitud entre palabras\n
  recibe(palabraAComparar, listaDePalabrasdeDondeComparar)\n
  return (palabra elegida,\n
          porcentaje de similitud,\n
          palabra mas cercana)
  """
  similar = 0
  contador = 0
  verbo_similar = ""
  for verbo in lista_verbos:
    contador += 1
    radio = jellyfish.jaro_winkler_similarity(palabra, verbo)
    if radio > similar:
      similar = radio
      verbo_similar = verbo
  if similar < 0.93:
    verbo_final = palabra
  else:
    verbo_final = verbo_similar
  return verbo_final

def tratamiento_texto(texto):
  """
  Se ingresa un texto y se devuelve texto:\n
  · En minuscula\n
  · Sin acentos\n
  · Sin signos de puntuacion
  """
  lista_palabras = []
  #texto = texto.lower()
  texto = unidecode(texto)
  espaciado = WhitespaceTokenizer()
  texto_espacios = espaciado.tokenize(texto)

  for palabra in texto_espacios:
    palabra_limpia = "".join([letra for letra in palabra if letra.isalnum()])
    lista_palabras.append(palabra_limpia)

  texto_limpio = " ".join([i for i in lista_palabras])
  return texto_limpio

#Función para reemplazar el final de una palabra por 'r'
def reemplazar_terminacion(palabra):
  """
  Intenta acercar los verbos conjugados lo mas posible a su forma en infinitivo
  · Remplaza las terminaciones "ar", "es", "me", "as", "te" por "r" y la terminacion "ste" por "ar"
  """
  lista_palabras = []
  espaciado = WhitespaceTokenizer()
  texto_espacios = espaciado.tokenize(palabra)

  for texto in texto_espacios:
    if texto[-3:] == "ste" and texto != "este":
      texto = texto[:-3] + "ar"
    elif (texto[-2:] == "es" or texto[-2:] == "me" or texto[-2:] == "as" or texto[-2:] == "te") and len(texto) > 3:
      texto = texto[:-2] + "r"
    lista_palabras.append(texto)

  texto_limpio = " ".join([i for i in lista_palabras])
  return texto_limpio.split()[0]

#Función para devolver los tokens normalizados del texto
def normalizar(texto):
  tokens=[]
  doc = nlp(texto)
  for t in doc:
    lemma=verbos_irregulares.get(t.text, t.lemma_.split()[0])
    lemma=re.sub(r'[^\w\s+\-*/]', '', lemma)
    if t.pos_ in ('VERB','PROPN','PRON','NOUN','AUX','SCONJ','ADJ','ADV','NUM') or lemma in lista_verbos:
      if t.pos_=='VERB':
        lemma = reemplazar_terminacion(lemma)
        tokens.append(raiz(tratamiento_texto(lemma)))
      else:
        tokens.append(tratamiento_texto(lemma))

  tokens = list(dict.fromkeys(tokens))
  tokens = list(filter(None, tokens))
  return tokens

# 4. Cargar bases de documentos

In [6]:
def carga_datos(ruta_carpeta):
  """
  Coloque la direccion de la carpeta con todos los datos a cargar
  return: diccionario con todas las direcciones de los archivos dentro de la carpeta
  """
  ruta_completa = []
  ruta_carpeta = ruta_carpeta
  ruta_txt = [ruta for ruta in os.listdir(ruta_carpeta)]
  ruta_txt
  for archivo in ruta_txt:
    ruta_completa.append(os.path.join(ruta_carpeta, archivo))
  return ruta_completa

def cargar_archivos(links):
  """
  Se cargan todos los datos de todos los archivos y los guarda en 3 listas, se les realiza un tratamientoa los datos
  return:preguntas, respuestas, categoria
  """
  lista_dialogos = []
  lista_dialogos_respuestas = []
  lista_tipo_dialogo = []
  for link in links:
    with open(link) as archivo:
      contador = 0
      for linea in archivo:
        if contador % 2 == 0:
          #linea = normalizacion(linea)
          linea = re.sub(r"[^\w\s+\-*/]", '', linea)
          linea = re.sub(r"[.*^\n]", '', linea)
          lista_dialogos.append(linea)
          lista_tipo_dialogo.append(os.path.split(link)[1][:-4])
        else:
          linea = re.sub(r"[^\w\s+\-*/]", '', linea)
          linea = re.sub(r"[.*^\n]", '', linea)
          lista_dialogos_respuestas.append(linea)
        contador += 1


  return lista_dialogos, lista_dialogos_respuestas, lista_tipo_dialogo

In [7]:
links = carga_datos("/content/drive/MyDrive/Chatbot/dialogos")
dialogos = cargar_archivos(links)

In [8]:
#Se crea el DataFrame y se le asigna cada lista obtenida a su respectiva columna
lista_df = {"dialogo": dialogos[0], "respuestas": dialogos[1], "tipo": dialogos[2], "interseccion": 0, "similarity": 0, "jaro_winkler": 0}
df_dialogo = pd.DataFrame(data= lista_df)


In [9]:
df_dialogo.sample(5)

,dialogo,respuestas,tipo,interseccion,similarity,jaro_winkler
456,hasta pronto nos vemos de nuevo pronto,Sí nos vemos de nuevo pronto Hasta pronto,Despedida,0,0,0
293,buenas tardes,Buenas tardes Me alegro de que estés aquí Qué ...,Saludos,0,0,0
19,Qué tan viejo eres,Como chatbot no tengo una edad física Soy una ...,Edad,0,0,0
243,hola como te va todo,Bastante bien gracias Necesitas ayuda con algo...,Saludos,0,0,0
821,puedes usar un chatbot con autogpt,Hola No soy ChatGPT me llamo Madre el chatbot ...,Otros,0,0,0


# 5. Buscar respuesta del Chatbot

In [10]:
def interseccion(usuario, pregunta_dialogo):
  interseccion = 0
  largo_palabra = 0
  dialogo = WhitespaceTokenizer().tokenize(pregunta_dialogo)
  for palabra_usuario in usuario:
    largo_palabra += 1
    for palabra_dialogo in dialogo:
      if palabra_usuario == palabra_dialogo:
        interseccion += 1
  porcentaje = interseccion / largo_palabra
  return round(porcentaje, 2)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def similarity(pregunta_usuario, pregunta_dialogo):
  vectorizer = TfidfVectorizer()
  vectorizer.fit_transform(df_dialogo["dialogo"])
  text1 = vectorizer.transform([pregunta_usuario])
  text2 = vectorizer.transform([pregunta_dialogo])
  return cosine_similarity(text1, text2)


In [11]:
#Función para verificar si el usuário inició un diálogo
def dialogo(consulta_usuario):
  consulta_normalizada = normalizar(consulta_usuario)
  df = df_dialogo.copy()
  consulta = " ".join(consulta_normalizada)
  for idx,row in df.iterrows():
    df.at[idx,'interseccion'] = interseccion(consulta_normalizada, row["dialogo"])
    df.at[idx,'similarity'] = similarity(consulta, row["dialogo"])[0][0]
    df.at[idx,'jaro_winkler'] = jellyfish.jaro_winkler_similarity(consulta, row["dialogo"])
    df.at[idx,'probabilidad'] = max(df.at[idx,'interseccion'],df.at[idx,'similarity'],df.at[idx,'jaro_winkler'])
  df.sort_values(by=['probabilidad','jaro_winkler'], inplace=True, ascending=False)
  probabilidad = df['probabilidad'].head(1).values[0]
  if probabilidad >= 0.93:
    print('Respuesta encontrada por el método de comparación de textos - Probabilidad: ', probabilidad)
    respuesta = df['respuestas'].head(1).values[0]
  else:
    print(probabilidad)
    respuesta = ''
  return respuesta



In [12]:
dialogo("Hola como estas")

0.9111111111111111


''

In [21]:
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [ ]:
#Normalizando las frases
label_encoder = LabelEncoder()
df_dialogo['palabras'] = df_dialogo['dialogo'].apply(lambda x: ' '.join(normalizar(x)))
df_dialogo['tipo_num'] = label_encoder.fit_transform(df_dialogo['tipo'])
df_dialogo = df_dialogo[df_dialogo.palabras.values!='']
df_dialogo

In [26]:
# Imprimir diccionario
relacion_diccionario = {}

# Iterar sobre las filas del DataFrame
for tipo, tipo_num in zip(df_dialogo['tipo'], df_dialogo['tipo_num']):
    relacion_diccionario[tipo_num] = tipo

# Imprimir el diccionario
print(relacion_diccionario)

{5: 'Edad', 15: 'Usuario', 11: 'Origen', 8: 'Funcion', 13: 'Saludos', 1: 'Aprendizaje', 2: 'Contacto', 4: 'Despedida', 9: 'Identidad', 3: 'Continuacion', 12: 'Otros', 10: 'Nombre', 14: 'Sentimiento', 6: 'ElProfeAlejo', 7: 'Error', 0: 'Agradecimiento'}


In [42]:
# Guardar el modelo entrenado
ruta_modelo = '/content/drive/MyDrive/Chatbot/modelo'
model.save_pretrained(ruta_modelo)
tokenizer.save_pretrained(ruta_modelo)

('/content/drive/MyDrive/Chatbot/modelo/tokenizer_config.json',
 '/content/drive/MyDrive/Chatbot/modelo/special_tokens_map.json',
 '/content/drive/MyDrive/Chatbot/modelo/vocab.txt',
 '/content/drive/MyDrive/Chatbot/modelo/added_tokens.json')

In [43]:
#Cargar el modelo entrenado
ruta_modelo = '/content/drive/MyDrive/Chatbot/modelo'
Modelo_TF = BertForSequenceClassification.from_pretrained(ruta_modelo)
tokenizer_TF = BertTokenizer.from_pretrained(ruta_modelo)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [32]:
# Calcular la precisión por clase
unique_classes = df_dialogo['tipo_num'].unique()

for class_label in unique_classes:
    # Filtrar los datos por clase
    class_data = df_dialogo[df_dialogo['tipo_num'] == class_label]

    # Preparar los datos de la clase para evaluar
    tokens = tokenizer_TF.batch_encode_plus(
        class_data['palabras'].tolist(),
        truncation=True,
        padding=True,
        return_tensors='pt'
    )

    inputs = tokens['input_ids']
    attention_mask = tokens['attention_mask']
    labels = class_data['tipo_num'].tolist()

    # Pasar los datos de la clase por el modelo
    with torch.no_grad():
        outputs = Modelo_TF(inputs, attention_mask=attention_mask)

    predicted_labels = outputs.logits.argmax(dim=1).tolist()

    # Calcular la precisión para la clase
    accuracy = accuracy_score(labels, predicted_labels)
    print(f"Precisión por clase {df_dialogo[df_dialogo.tipo_num == class_label]['tipo'].unique()[0]}: {accuracy}")

Precisión por clase Edad: 0.8387096774193549
Precisión por clase Usuario: 0.7297297297297297
Precisión por clase Origen: 0.78
Precisión por clase Funcion: 0.9315068493150684
Precisión por clase Saludos: 0.9777777777777777
Precisión por clase Aprendizaje: 0.9069767441860465
Precisión por clase Contacto: 0.6206896551724138
Precisión por clase Despedida: 0.9629629629629629
Precisión por clase Identidad: 0.9876543209876543
Precisión por clase Continuacion: 0.8709677419354839
Precisión por clase Otros: 0.9753846153846154
Precisión por clase Nombre: 1.0
Precisión por clase Sentimiento: 0.971830985915493
Precisión por clase ElProfeAlejo: 0.8148148148148148
Precisión por clase Error: 0.7916666666666666
Precisión por clase Agradecimiento: 0.9242424242424242


In [33]:
# Procesar nueva frase
frase = ' '.join(normalizar('donde vives?'))

# Tokenizar la frase de entrada
tokens = tokenizer_TF.encode_plus(
    frase,
    add_special_tokens=True,
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# Obtener los input_ids y attention_mask
input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']

# Realizar la predicción
with torch.no_grad():
    outputs = Modelo_TF(input_ids, attention_mask)

# Obtener las etiquetas predichas
etiquetas_predichas = torch.argmax(outputs.logits, dim=1)

# Decodificar las etiquetas predichas
etiquetas_decodificadas = etiquetas_predichas.tolist()

diccionario = {14: 'Sentimiento', 13: 'Saludos', 10: 'Nombre', 9: 'Identidad', 6: 'ElProfeAlejo', 1: 'Aprendizaje', 8: 'Funcion', 15: 'Usuario', 11: 'Origen', 5: 'Edad', 0: 'Agradecimiento', 3: 'Continuacion', 2: 'Contacto', 4: 'Despedida', 12: 'Otros', 7: 'Error'}
llave_buscada = etiquetas_decodificadas[0]
clase_encontrada = diccionario[llave_buscada]
print("La frase", frase, "se clasifica como: ", clase_encontrada)

La frase donde vivir se clasifica como:  Contacto


In [34]:
#Función para dialogar utilizando el modelo
def clasificacion_modelo(pregunta):
  frase = ' '.join(normalizar(pregunta))
  # Tokenizar la frase de entrada
  tokens = tokenizer_TF.encode_plus(
      frase,
      add_special_tokens=True,
      max_length=128,
      padding='max_length',
      truncation=True,
      return_tensors='pt'
  )

  # Obtener los input_ids y attention_mask
  input_ids = tokens['input_ids']
  attention_mask = tokens['attention_mask']

  # Realizar la predicción
  with torch.no_grad():
      outputs = Modelo_TF(input_ids, attention_mask)

  # Obtener las etiquetas predichas
  etiquetas_predichas = torch.argmax(outputs.logits, dim=1)

  # Decodificar las etiquetas predichas
  etiquetas_decodificadas = etiquetas_predichas.tolist()

  diccionario = {14: 'Sentimiento', 13: 'Saludos', 10: 'Nombre', 9: 'Identidad', 6: 'ElProfeAlejo', 1: 'Aprendizaje', 8: 'Funcion', 15: 'Usuario', 11: 'Origen', 5: 'Edad', 0: 'Agradecimiento', 3: 'Continuacion', 2: 'Contacto', 4: 'Despedida', 12: 'Otros', 7: 'Error'}
  llave_buscada = etiquetas_decodificadas[0]
  clase_encontrada = diccionario[llave_buscada]

  return clase_encontrada


  #Buscar respuesta más parecida en la clase encontrada
  df = df_dialogo[df_dialogo['tipo'] == clase_encontrada]
  df.reset_index(inplace=True)
  vectorizer = TfidfVectorizer()
  dialogos_num = vectorizer.fit_transform(df['dialogo'])
  pregunta_num = vectorizer.transform([tratamiento_texto(pregunta)])
  similarity_scores = cosine_similarity(dialogos_num, pregunta_num)
  indice_pregunta_proxima = similarity_scores.argmax()

  if max(similarity_scores)>0.5 and clase_encontrada not in ['Otros']:
    print('Respuesta encontrada por el modelo Transformers - tipo:',clase_encontrada)
    respuesta = df['respuesta'][indice_pregunta_proxima]
  else:
    respuesta = ''
  return respuesta

#Función para devolver una respuesta final buscada en todos los métodos disponibles
def respuesta_chatbot(pregunta):
  respuesta = dialogo(pregunta)
  if respuesta != '':
    return respuesta
  else:
    respuesta = clasificacion_modelo(pregunta)
    if respuesta != '':
      return respuesta
    else:
      return 'Respuesta no encontrada'

# 6. Ejecutar Chatbot

In [41]:
pregunta='quien eres?'
respuesta = respuesta_chatbot(pregunta)
print(respuesta)

Respuesta encontrada por el método de comparación de textos - Probabilidad:  0.9577777777777777
Soy un chatbot me llamo Madre y tú cómo te llamas


#Modelo Transformers

In [27]:
# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
df_train, df_test = train_test_split(df_dialogo, test_size=0.2, random_state=42)

# Cargar el modelo preentrenado de BERT para clasificación en español
model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=df_dialogo['tipo_num'].nunique())
tokenizer = BertTokenizer.from_pretrained(model_name)

# Tokenizar y codificar las frases de entrenamiento
train_inputs = tokenizer.batch_encode_plus(
    df_train['palabras'].tolist(),
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# Tokenizar y codificar las frases de prueba
test_inputs = tokenizer.batch_encode_plus(
    df_test['palabras'].tolist(),
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# Preparar los datos de entrenamiento y prueba
train_data = torch.utils.data.TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], torch.tensor(df_train['tipo_num'].tolist()))
test_data = torch.utils.data.TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], torch.tensor(df_test['tipo_num'].tolist()))

# Definir el optimizador y la función de pérdida
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Entrenamiento del modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)

for epoch in range(5):  # Número de épocas de entrenamiento
    total_loss = 0

    for batch in train_dataloader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print("Epoch:", epoch + 1, "Loss:", total_loss)

# Evaluación del modelo
model.eval()
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

with torch.no_grad():
    predictions = []
    true_labels = []

    for batch in test_dataloader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        outputs = model(input_ids, attention_mask=attention_mask)

        _, predicted_labels = torch.max(outputs.logits, dim=1)

        predictions.extend(predicted_labels.tolist())
        true_labels.extend(labels.tolist())

accuracy = accuracy_score(true_labels, predictions)
print("Precisión:", accuracy)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1 Loss: 136.42086732387543
Epoch: 2 Loss: 108.83632934093475
Epoch: 3 Loss: 84.38828194141388
Epoch: 4 Loss: 61.31737670302391
Epoch: 5 Loss: 42.479231268167496
Precisión: 0.8260869565217391
